In [1]:
import numpy as np
from scipy.optimize import linprog

# Función para resolver el problema de forma relajada (sin requerir enteros)
def solve_relaxed(c, A, b):
    """
    Resuelve el problema de programación lineal relajada (sin requerir enteros).
    :param c: Coeficientes de la función objetivo.
    :param A: Matriz de restricciones.
    :param b: Límite de las restricciones.
    :return: Resultado de linprog.
    """
    result = linprog(c=-np.array(c), A_ub=A, b_ub=b, method="highs")
    return result

# Función de Branch and Bound
def branch_and_bound(c, A, b):
    """
    Aplica el método Branch and Bound para resolver un problema de programación entera.
    :param c: Coeficientes de la función objetivo.
    :param A: Matriz de restricciones.
    :param b: Límite de las restricciones.
    :return: Mejor solución entera encontrada.
    """
    best_solution = None  # Mejor solución factible encontrada
    best_value = float('-inf')  # Mejor valor de la función objetivo

    # Pila para manejar los nodos del árbol de búsqueda
    stack = [(A, b)]

    while stack:
        A_current, b_current = stack.pop()  # Sacamos un nodo a procesar

        # Resolver el problema relajado
        result = solve_relaxed(c, A_current, b_current)

        if not result.success:  # Si no tiene solución, continuamos
            continue

        x_relaxed = result.x  # Solución continua (relajada)
        z_relaxed = -result.fun  # Valor de la función objetivo

        # Si el valor es menor que el mejor encontrado, descartamos
        if z_relaxed <= best_value:
            continue

        # Si todos los valores son enteros, es una solución candidata
        if np.all(np.floor(x_relaxed) == x_relaxed):
            if z_relaxed > best_value:
                best_solution = x_relaxed
                best_value = z_relaxed
            continue

        # Encontramos la variable fraccional más significativa para ramificar
        idx = np.argmax(x_relaxed - np.floor(x_relaxed))
        x_fractional = x_relaxed[idx]

        # Crear restricciones adicionales para dividir el problema en dos ramas
        new_constraint1 = np.zeros_like(c)
        new_constraint1[idx] = 1  # x[idx] <= floor(x_fractional)

        new_constraint2 = np.zeros_like(c)
        new_constraint2[idx] = -1  # x[idx] >= ceil(x_fractional)

        # Agregar las nuevas ramas a la pila
        stack.append((np.vstack([A_current, new_constraint1]), np.append(b_current, np.floor(x_fractional))))
        stack.append((np.vstack([A_current, new_constraint2]), np.append(b_current, -np.ceil(x_fractional))))

    return best_solution, best_value

# Definir coeficientes de la función objetivo
c = [3, 5]

# Definir restricciones
A = [[2, 3], [1, 2]]  # Coeficientes de las restricciones
b = [6, 4]  # Límites de las restricciones

# Resolver usando Branch and Bound
solution, value = branch_and_bound(c, A, b)

print("Mejor solución entera encontrada:", solution)
print("Valor óptimo:", value)


Mejor solución entera encontrada: [0. 2.]
Valor óptimo: 10.0
